In [4]:
# Dataset: BBC news
%run -i "../util/lang_utils.ipynb"

In [5]:
from datasets import load_dataset
from nltk import word_tokenize
from math import ceil
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords


c:\Users\ravik\anaconda3\envs\nlp_cookbook\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
train_dataset = load_dataset("SetFit/bbc-news", split="train")
test_dataset = load_dataset("SetFit/bbc-news", split="test")
train_df = train_dataset.to_pandas()
test_df = test_dataset.to_pandas()
print(train_df)
print(test_df)

                                                   text  label     label_text
0     wales want rugby league training wales could f...      2          sport
1     china aviation seeks rescue deal scandal-hit j...      1       business
2     rock band u2 break ticket record u2 have smash...      3  entertainment
3     markets signal brazilian recovery the brazilia...      1       business
4     tough rules for ringtone sellers firms that fl...      0           tech
...                                                 ...    ...            ...
1220  us economy shows solid gdp growth the us econo...      1       business
1221  microsoft releases bumper patches microsoft ha...      0           tech
1222  stuart joins norwich from addicks norwich have...      2          sport
1223  why few targets are better than many the econo...      1       business
1224  boothroyd calls for lords speaker betty boothr...      4       politics

[1225 rows x 3 columns]
                                       

In [7]:
from datasets import load_dataset

train_dataset = load_dataset("SetFit/bbc-news", split="train")
test_dataset = load_dataset("SetFit/bbc-news", split="test")
train_df = train_dataset.to_pandas()
test_df = test_dataset.to_pandas()
print(train_df)

                                                   text  label     label_text
0     wales want rugby league training wales could f...      2          sport
1     china aviation seeks rescue deal scandal-hit j...      1       business
2     rock band u2 break ticket record u2 have smash...      3  entertainment
3     markets signal brazilian recovery the brazilia...      1       business
4     tough rules for ringtone sellers firms that fl...      0           tech
...                                                 ...    ...            ...
1220  us economy shows solid gdp growth the us econo...      1       business
1221  microsoft releases bumper patches microsoft ha...      0           tech
1222  stuart joins norwich from addicks norwich have...      2          sport
1223  why few targets are better than many the econo...      1       business
1224  boothroyd calls for lords speaker betty boothr...      4       politics

[1225 rows x 3 columns]


In [7]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=2, max_df=0.95)
vectorizer.fit(train_df["text"])

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,analyzer,'word'
,stop_words,'english'
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [8]:
def sort_data_tfidf_score(coord_matrix):
    tuples = zip(coord_matrix.col, coord_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)    

In [9]:
def get_keyword_strings(vectorizer, num_words, sorted_vector):
    words = []
    index_dict = vectorizer.get_feature_names_out()
    for (item_index, score) in sorted_vector[0:num_words]:
        word = index_dict[item_index]
        words.append(word)
    return words

In [10]:
def get_keywords_simple(vectorizer, input_text, num_output_words=10):
    vector = vectorizer.transform([input_text])
    sorted = sort_data_tfidf_score(vector.tocoo())
    words = get_keyword_strings(vectorizer, num_output_words, sorted)
    return words

In [11]:
print(test_df.iloc[0]["text"])
keywords = get_keywords_simple(vectorizer, test_df.iloc[0]["text"])
print(keywords)

carry on star patsy rowlands dies actress patsy rowlands  known to millions for her roles in the carry on films  has died at the age of 71.  rowlands starred in nine of the popular carry on films  alongside fellow regulars sid james  kenneth williams and barbara windsor. she also carved out a successful television career  appearing for many years in itv s well-loved comedy bless this house. rowlands died in hove on saturday morning  her agent said.  born in january 1934  rowlands won a scholarship to the guildhall school of speech and drama scholarship when she was just 15.  after spending several years at the players theatre in london  she made her film debut in 1963 in tom jones  directed by tony richardson. she made her first carry on film in 1969 where she appeared in carry on again doctor. rowlands played the hard-done-by wife or the put-upon employee as a regular carry on star. she also appeared in carry on at your convenience  carry on matron and carry on loving  as well as othe

In [12]:
stop_words = list(stopwords.words('english'))
stop_words.remove("the")
trigram_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2, ngram_range=(1,3), max_df=0.95)
trigram_vectorizer.fit(train_df["text"])

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,analyzer,'word'
,stop_words,"['a', 'about', ...]"
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [13]:
def get_keyword_strings_all(vectorizer, sorted_vector):
    words = []
    index_dict = vectorizer.get_feature_names_out()
    for (item_index, score) in sorted_vector:
        word = index_dict[item_index]
        words.append(word)
    return words

In [14]:
keywords = get_keywords_simple(vectorizer, test_df.iloc[0]["text"])
print(keywords)

['carry', 'theatre', 'scholarship', 'appeared', 'films', 'mrs', 'agent', 'drama', 'died', 'school']


In [16]:
def get_keywords_complex(vectorizer, input_text, spacy_model, num_words=70):
    keywords = []
    doc = spacy_model(input_text)
    vector = vectorizer.transform([input_text])

    sorted_items = sort_data_tfidf_score(vector.tocoo())
    ngrams = get_keyword_strings_all(vectorizer, sorted_items)

    ents = [ent.text.lower() for ent in doc.noun_chunks]

    for keyword in ngrams[:num_words]:
        if keyword.lower() in ents and not keyword.isdigit() and keyword not in keywords:
            keywords.append(keyword)

    return keywords


In [17]:
def get_keywords_simple(vectorizer, input_text,
                          num_output_words=10):
    vector = vectorizer.transform([input_text])
    sorted = sort_data_tfidf_score(vector.tocoo())
    words = get_keyword_strings(vectorizer, num_output_words, sorted)
    return words

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

trigram_vectorizer = TfidfVectorizer(
    ngram_range=(1, 3),
    stop_words="english",
    max_features=5000
)

trigram_vectorizer.fit(train_df["text"])


,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,analyzer,'word'
,stop_words,'english'
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [19]:
keywords = get_keywords_complex(trigram_vectorizer, test_df.iloc[0]["text"], small_model)
print(keywords)

['carry', 'films', 'stage', 'star', 'film', 'london', 'directors', 'skills', 'recent years', 'actress', 'millions', 'speech']
